In [1]:
import numpy as np
import scipy.stats as spst
import scipy.optimize as spop
import scipy.interpolate as spint
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn import datasets, linear_model
from scipy import stats
import textwrap
from scipy.interpolate import interp1d


#  Pefil CTD1 (XBT)

In [125]:
est = input('Nome da Estacao:')
num = input('Numero da Estacao:')
loc = r'C:\Users\Apaluz\programacao\pauloqrepo\jupyter\Analises\CTDxCTD\perfil_'+num+r'\xbt_'+num+'.xlsx'
df = pd.read_excel(loc)
df.head(5)

Nome da Estacao:MCT030
Numero da Estacao:30


,prof_xbt,temp_xbt
0,4.917,14.4570
1,5.919,14.4613
2,6.968,14.4902
3,7.974,14.5423
4,8.959,14.5630


In [ ]:
plt.figure(figsize=(6, 8))
plt.plot(df['temp_xbt'],df['prof_xbt'], label='Dados observados')
plt.title('Perfil XBT Estação'+' '+est, fontsize = 20)
plt.xlabel('Temperatura[°C]', fontsize = 15)
plt.ylabel('Profundidade', fontsize = 15)
plt.ylim(600, 0)
plt.legend(loc='lower left', shadow=True, fontsize='large')
plt.show()
#plt.savefig(r'C:\Users\Apaluz\programacao\pauloqrepo\jupyter\regressaolinear\ANALISE\perfil_'+num+r'\xbt_'+num+'.png')

# Perfil CTD2 (CTDMB) 

In [126]:
local = r'C:\Users\Apaluz\programacao\pauloqrepo\jupyter\Analises\CTDxCTD\perfil_'+num+r'\ctd_'+num+'.xlsx'
df2 = pd.read_excel(local)
df2.head(5)

,prof_ctd,temp_ctd
0,3,14.4929
1,4,14.4927
2,5,14.4929
3,6,14.5058
4,7,14.5141


In [5]:
plt.figure(figsize= (6,8))
plt.plot(df2['temp_ctd'], df2['prof_ctd'], 'g',label='Dados observados')
plt.title('Perfil CTD Estação'+' '+est, fontsize = 20)
plt.xlabel('Temperatura [ITS-90, deg C]', fontsize = 15)
plt.ylabel('Profundidade [salt water, m]', fontsize = 15)
plt.ylim(600, 0)
plt.legend(loc='lower left', shadow=True, fontsize='large')
plt.show()

# CTD x XBT

In [127]:
plt.figure(figsize=(6, 8))
plt.plot(df['temp_xbt'],df['prof_xbt'], 'b', label="Perfil XBT", linewidth=5)
plt.plot(df2['temp_ctd'], df2['prof_ctd'], 'r', label="Perfil CTD", linewidth=5)
plt.ylim(600, 0)
plt.xlim(0, 25)
plt.title('Perfil XBT x CTD Estação'+' '+est, fontsize = 20)
plt.xlabel('Temperatura[°C]', fontsize = 15)
plt.ylabel('Profundidade[m]', fontsize = 15)
plt.legend(loc='lower left', shadow=True, fontsize='large')
plt.show()

# Interpolacao

Primeiro tratar os dados para o comecar nas mesmas profundidades e terminar nas respectivas. (Evitar os NaN s) 


In [128]:
localint = r'C:\Users\Apaluz\programacao\pauloqrepo\jupyter\Analises\CTDxCTD\perfil_'+num+r'\xbt_ctd_'+num+'.xlsx'
dfint = pd.read_excel(localint)
dfint.head(4)

,prof_xbt,temp_xbt,prof_ctd,temp_ctd
0,4.917,14.4570,4,14.4927
1,5.919,14.4613,5,14.4929
2,6.968,14.4902,6,14.5058
3,7.974,14.5423,7,14.5141


In [129]:
dff = []
for i in dfint['temp_ctd']:
    dff.append(i)
dff2 = []
for i in dfint['prof_ctd']:
    dff2.append(i)
dfint3 = pd.DataFrame(data = dff2, columns = ['prof_ctd'])
dfint3['temp_ctd'] = dff

dfint33 = dfint3.dropna()

#dfint33.to_excel(r'C:\Users\Apaluz\programacao\pauloqrepo\jupyter\regressaolinear\ANALISE\perfil_13\ctd_sem_Nan.xlsx',index=False,header=False)

In [130]:
x = dfint['prof_xbt']
y = dfint['temp_xbt']
yi = dfint33['prof_ctd']
xi = np.interp(yi, x, y)
plt.figure()
plt.plot(dfint33['temp_ctd'] , -dfint33['prof_ctd'] , '-ob', label = ' CTD')
plt.plot(xi, -yi, '-or', label = 'XBT interpolado')
plt.plot(dfint['temp_xbt'] , -dfint['prof_xbt'], '-og', label = ' XBT')
plt.legend(loc='upper left', shadow=True, fontsize='large')
#plt.ylim(600, 0)
plt.show()

In [131]:
a = len(xi)
b = len(x)
c = len(yi)
d = len(y)
if a < b:
    print('CORRETO, CONTINUE!!!')
print('xi =',a,'; x =',b,'; yi =', c,'; y =',d)

xi = 58 ; x = 58 ; yi = 58 ; y = 58


In [132]:
dfinterpolado = pd.DataFrame(data = xi, columns = ['xbt_inter'])
dfinterpolado.head(5)

,xbt_inter
0,14.457000
1,14.457356
2,14.463532
3,14.491857
4,14.542846


# Regressao Linear

ctd = dfint33 e xbt interpolado = dfinterpolado

In [133]:
dfinterpolado['ctd'] = dfint33['temp_ctd']
dfinterpolado.head(5)


,xbt_inter,ctd
0,14.457000,14.4927
1,14.457356,14.4929
2,14.463532,14.5058
3,14.491857,14.5141
4,14.542846,14.5214


In [134]:
dfinterpolado.describe()

,xbt_inter,ctd
count,58.000000,58.000000
mean,14.943067,14.917272
std,0.213708,0.234249
min,14.457000,14.492700
25%,14.899318,14.670000
50%,15.035956,14.999600
75%,15.092360,15.100025
max,15.178383,15.177700


In [135]:
minixbt = dfinterpolado['xbt_inter'].min()
minictd = dfinterpolado['ctd'].min()
maxixbt = dfinterpolado['xbt_inter'].max()
maxictd = dfinterpolado['ctd'].max()

npMatrix = np.matrix(dfinterpolado)
X, Y = npMatrix[:,1], npMatrix[:,0]
mdl = LinearRegression().fit(X,Y) # either this or the next line
#mdl = LinearRegression().fit(filtered_data[['x']],filtered_data.y)
m = mdl.coef_[0]
b = mdl.intercept_
#print("formula: y = {0}x + {1}".format(m, b)) # following slope intercept form 
stm = str(m)
stb = str(b)
form = ("y = {0}x + {1}".format(stm[1:len(stm)-1], stb[1:len(stb)-1]))
print(form)

y =  0.82902802x +  2.5762297


In [136]:
##Estatistica, regressao linear coeficientes
x = dfinterpolado["ctd"]
y = dfinterpolado["xbt_inter"]
slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
a = slope
bb = intercept
rr = r_value**2
print(rr)

0.825754318393


In [147]:
foorm = str(form) +' '+ 'R² ='+' '+ str(rr)
lege = textwrap.fill(foorm,len(str(form)))
palavra = 'Comparação dos valores de Temperatura CTD MB x CTD FURG' +' '+est 
legen = textwrap.fill(palavra,40)
print(legen)

plt.figure(figsize=(9, 10))
plt.scatter(X,Y, color='r', s=50, label='Dados Observados')
#plt.plot(X,Y,'og', s=4, label='Dados Observados')
plt.plot([0,50],[b,m*50+b],'-b',linewidth=4, label=lege)
plt.title(legen, fontsize = 20)
plt.xlabel('CTD Observado', fontsize = 15)
plt.ylabel('XBT Interpolado', fontsize = 15)
plt.xlim(minixbt - 1,maxixbt + 1)
plt.ylim(minictd - 1,maxictd + 1)
plt.legend(loc='upper left', shadow=True, fontsize='large')
plt.grid()
plt.show()

Comparação dos valores de Temperatura
CTD MB x CTD FURG Total


In [138]:
dados = pd.DataFrame(data = X, columns= ['CTD_observado'])
dados['XBT_interpolado'] = Y 
dados.to_excel(r'C:\Users\Apaluz\programacao\pauloqrepo\jupyter\Analises\CTDxCTD\perfil_'+ num +r'\perfil_REGRE'+ num +'.xlsx',index=True,header=True)



# Regressao e Perfis

In [29]:
palavras = 'Perfil CTD MB x CTD FURG Estação'+' '+est
legens = textwrap.fill(palavras,25)
print(legens)

Perfil CTD MB x CTD FURG
Estação MCT007


In [142]:
palavras = 'Perfil CTD MB x CTD FURG Estação'+' '+est
legens = textwrap.fill(palavras,25)
print(legens)

plt.figure(figsize=(6, 8))
plt.plot(df['temp_xbt'],df['prof_xbt'], 'b', label="Perfil CTD FURG", linewidth=5)
plt.plot(df2['temp_ctd'], df2['prof_ctd'], 'r', label="Perfil CTD MB", linewidth=5)
plt.ylim(80, 0)
plt.xlim(10, 20)
plt.title(legens, fontsize = 20)
plt.xlabel('Temperatura[°C]', fontsize = 15)
plt.ylabel('Profundidade[m]', fontsize = 15)
plt.legend(loc='lower left', shadow=True, fontsize='large')
plt.grid()
#plt.show()

plt.figure(figsize=(9, 10))
plt.scatter(X,Y, color='r', s=50, label='Dados Observados')
#plt.plot(X,Y,'og', s=4, label='Dados Observados')
plt.plot([0,50],[b,m*50+b],'-b',linewidth=4, label=lege)
plt.title(legen, fontsize = 20)
plt.xlabel('CTD MB', fontsize = 15)
plt.ylabel('CTD FURG', fontsize = 15)
plt.xlim(minixbt - 1,maxixbt + 1)
plt.ylim(minictd - 1,maxictd + 1)
plt.legend(loc='upper left', shadow=True, fontsize='large')
plt.grid()
plt.show()

Perfil CTD MB x CTD FURG
Estação MCT030


# REGRESSAO TOTAL

In [144]:
#####regressao Total

regtotal = pd.read_excel(r'C:\Users\Apaluz\programacao\pauloqrepo\jupyter\Analises\CTDxCTD\REGRE_TOTAL_CTD_CTD.xlsx')
regtotal.head(4)

,CTD_observado,XBT_interpolado
0,14.4927,14.457000
1,14.4929,14.457356
2,14.5058,14.463532
3,14.5141,14.491857


In [148]:
minixbt = regtotal['XBT_interpolado'].min()
minictd = regtotal['CTD_observado'].min()
maxixbt = regtotal['XBT_interpolado'].max()
maxictd = regtotal['CTD_observado'].max()

npMatrix = np.matrix(regtotal)
XX, YY = npMatrix[:,1], npMatrix[:,0]
mdl = LinearRegression().fit(X,Y) # either this or the next line
#mdl = LinearRegression().fit(filtered_data[['x']],filtered_data.y)
m = mdl.coef_[0]
b = mdl.intercept_
#print("formula: y = {0}x + {1}".format(m, b)) # following slope intercept form 
stm = str(m)
stb = str(b)
form = ("y = {0}x + {1}".format(stm[1:len(stm)-1], stb[1:len(stb)-1]))
print(form)

x = regtotal['CTD_observado']
y = regtotal['XBT_interpolado']
slope, intercept, r_value, p_value, std_err = stats.linregress(x,y)
a = slope
bb = intercept
rr = r_value**2
print(rr)

plt.figure(figsize=(9, 10))
plt.scatter(XX,YY, color='r', s=50, label='Dados Observados')
#plt.plot(X,Y,'og', s=4, label='Dados Observados')
plt.plot([0,50],[b,m*50+b],'-b',linewidth=4, label=lege)
plt.title(legen, fontsize = 20)
plt.xlabel('CTD Observado', fontsize = 15)
plt.ylabel('XBT Interpolado', fontsize = 15)
plt.xlim(minixbt - 1,maxixbt + 1)
plt.ylim(minictd - 1,maxictd + 1)
plt.legend(loc='upper left', shadow=True, fontsize='large')
plt.grid()
plt.show()

y =  0.82902802x +  2.5762297
0.996567832534
